Para los temas de ecuaciones de einstein. Añadimos el cálculo del tensor de Einstein

In [1]:
import sympy as smp
from itertools import product

def computar(variables_str, g):
    """
    Con la métrica como una matriz de strings, calcular simbolos de christoffel, etc
    Modificado para que funcione en cualquier número de dimensiones
    """
    tau = smp.symbols("tau")

    variables = smp.symbols(variables_str, cls=smp.Function)
    variables = [v(tau) for v in variables]

    n = len(variables)
    
    for i, j in product(range(0, n), repeat=2):
        g[i][j] = smp.sympify(g[i][j])

    L = smp.sympify(0)

    for i, j in product(range(0, n), repeat=2):
        L += g[i][j]*smp.diff(variables[i], tau)*smp.diff(variables[j], tau)
    
    # Estas tres lineas toman algo de tiempo
    eel = smp.euler_equations(L, variables, tau)

    Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz n*n*n

    for i in range(0, n): # primer indice
        # Resolvemos d^2/dt^2 (v^i) = elem
        elem = smp.solve(eel[i], smp.diff(variables[i], tau, tau))[0]
        elem = smp.expand(elem)
        # elem será una suma de coeficientes lineares en las derivadas de las variables_tau
        # Por ahora hacemos un loop doble, pero tal vez sería posible extraer las derivadas directamente
        # https://docs.sympy.org/latest/index.html
        for j, k in product(range(0, n), repeat=2):
            var = smp.diff(variables[j], tau)*smp.diff(variables[k], tau)
            Gamma[i][j][k] = -elem.coeff(var)
            if j != k: # The common pitfall
                Gamma[i][j][k] = Gamma[i][j][k]/2
                    
    Riemman = [[[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz n*n*n*n

    for i, j, k, l in product(range(0, n), repeat=4):
        Riemman[i][j][k][l] = smp.diff(Gamma[i][l][j], variables[k]) - smp.diff(Gamma[i][k][j], variables[l])
        for alpha in range(0, n):
            Riemman[i][j][k][l] += Gamma[i][k][alpha]*Gamma[alpha][l][j] - Gamma[i][l][alpha]*Gamma[alpha][k][j]
        # https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html#trigsimp
        Riemman[i][j][k][l] = smp.simplify(Riemman[i][j][k][l])
        # Riemman[i][j][k][l] = smp.expand_trig(Riemman[i][j][k][l])
        # Riemman[i][j][k][l] = smp.trigsimp(Riemman[i][j][k][l])

    Ricci = [[0 for _ in range(n)] for _ in range(n)] # Matriz n*n

    for i, j in product(range(0, n), repeat=2):
        for alpha in range(0, n):
            Ricci[i][j] += Riemman[alpha][i][alpha][j]
        Ricci[i][j] = smp.simplify(Ricci[i][j])

    R = smp.sympify(0)

    # g_inversa = inverso_analitico(g)

    for i, j in product(range(0, n), repeat=2):
        R += (smp.Matrix(g)**(-1)).row(i)[j]*Ricci[i][j]
        # R += g_inversa[i][j]*Ricci[i][j]

    R = smp.simplify(R)
    R = smp.expand_trig(R)
    R = smp.trigsimp(R)
    
    E = [[0 for _ in range(n)] for _ in range(n)] # Matriz n*n

    for i, j in product(range(0, n), repeat=2):
        E[i][j] = smp.simplify(Ricci[i][j] - smp.Rational(1, 2)*R*g[i][j])

    return Gamma, Riemman, Ricci, R, E

def calcular_metrica(variables_str, cambios_str, as_strings=True):
    """
    Calcula la métrica de un cambio de coordenadas aleatorio en el espacio plano. Necesita las nuevas coordenadas y las 
    viejas coordenadas en función de las nuevas. 
    Ejemplo: pasar de espacio plano con (x, y, z) a espacio plano con (r, theta, phi) y una nueva métrica se hace con:
    calcular_metrica(["r", "theta", "phi"], ["r*sin(theta)*cos(phi)", "r*sin(theta)*sin(phi)", "r*cos(theta)"])
    """
    n = len(cambios_str)
    g = [[0 for _ in range(0, n)] for _ in range(0, n)] # Matriz n*n

    cambio = [smp.sympify(v) for v in cambios_str]
    variables = smp.symbols(variables_str)

    for i, j in product(range(0, n), repeat=2):
        for v in cambio:
            # Implicitamente g_(ij) de la que venimos es la delta de kroneker
            g[i][j] += smp.diff(v, variables[i])*smp.diff(v, variables[j])
        g[i][j] = g[i][j].simplify()

    # Cambiamos las variables por variables que dependen explicitamente de tau    
    tau = smp.symbols("tau")
    variables_tau = smp.symbols(variables_str, cls=smp.Function)
    variables_tau = [v(tau) for v in variables_tau]

    for i, j in product(range(0, n), repeat=2):
        for k in range(0, n):
            g[i][j] = g[i][j].subs(variables[k], variables_tau[k])

    if as_strings:
        return [[str(g[i][j]) for j in range(0, n)] for i in range(0, n)]
    
    return g

kroneker = lambda i, j: 1 if i == j else 0

def inverso_analitico(M): 
    """
    Calcula el inverso de la matriz M de forma analítica con la descomposición de Cayley-Hamilton
    https://en.wikipedia.org/wiki/Invertible_matrix#Analytic_solution
    Sólo implementado hasta orden 3
    """

    m = len(M)
    M_inv = [[0 for _ in range(0, m)] for _ in range(0, m)]

    smp_M = smp.Matrix(M)

    traza, determinante = smp.trace(smp_M), smp.Determinant(smp_M).doit()

    if determinante == 0: raise ValueError("La matriz dada no tiene inverso")

    if m == 2:
        for i, j in product(range(0, m), repeat=2):
            M_inv[i][j] = (1/determinante)*(traza*kroneker(i, j) - M[i][j])
        return M_inv
    if m == 3:
        traza_cuad = smp.trace(smp_M**2)
        for i, j in product(range(0, m), repeat=2):
            M_inv[i][j] = (1/determinante)*((1/2)*(traza**2 - traza_cuad**2)*kroneker(i, j) - M[i][j]*traza + (smp_M**2).row(i)[j])
        return M_inv
    
def computar_e_imprimir(variables_str:list, g):
    n = len(variables_str)

    Gamma, Riemman, Ricci, R, E = computar(variables_str, g)

    v = variables_str
        
    for i, j, k in product(range(0, n), repeat=3):
        if Gamma[i][j][k] != 0:
            s = str(Gamma[i][j][k]).replace("(tau)", "")
            print(f"Gamma^{v[i]}_({v[j]} {v[k]}) = {s}")
    print()
    for i, j, k, l in product(range(0, n), repeat=4):
        if Riemman[i][j][k][l] != 0:
            s = str(Riemman[i][j][k][l]).replace("(tau)", "")
            print(f"Riemman^{v[i]}_({v[j]} {v[k]} {v[l]}) = {s}")
    print()
    for i, j in product(range(0, n), repeat=2):
        if Ricci[i][j] != 0:
            s = str(Ricci[i][j]).replace("(tau)", "")
            print(f"Ricci_({v[i]} {v[j]}) = {s}")
    print()
    print("R =", str(R).replace("(tau)", ""))
    print()
    for i, j in product(range(0, n), repeat=2):
        if E[i][j] != 0:
            s = str(E[i][j]).replace("(tau)", "")
            print(f"E_({v[i]} {v[j]}) = {s}")

In [2]:
# comprobación ejercicio 1
variables = ["t", "x"]

g = [
    ["-E^(2*f(x(tau)))", "0"],
    ["0", "E^(2*g(x(tau)))"]
]

computar_e_imprimir(variables, g)

Gamma^t_(t x) = Derivative(f(x), x)
Gamma^t_(x t) = Derivative(f(x), x)
Gamma^x_(t t) = exp(2*f(x))*exp(-2*g(x))*Derivative(f(x), x)
Gamma^x_(x x) = Derivative(g(x), x)

Riemman^t_(x t x) = -Derivative(f(x), x)**2 + Derivative(f(x), x)*Derivative(g(x), x) - Derivative(f(x), (x, 2))
Riemman^t_(x x t) = Derivative(f(x), x)**2 - Derivative(f(x), x)*Derivative(g(x), x) + Derivative(f(x), (x, 2))
Riemman^x_(t t x) = (-Derivative(f(x), x)**2 + Derivative(f(x), x)*Derivative(g(x), x) - Derivative(f(x), (x, 2)))*exp(2*f(x) - 2*g(x))
Riemman^x_(t x t) = (Derivative(f(x), x)**2 - Derivative(f(x), x)*Derivative(g(x), x) + Derivative(f(x), (x, 2)))*exp(2*f(x) - 2*g(x))

Ricci_(t t) = (Derivative(f(x), x)**2 - Derivative(f(x), x)*Derivative(g(x), x) + Derivative(f(x), (x, 2)))*exp(2*f(x) - 2*g(x))
Ricci_(x x) = -Derivative(f(x), x)**2 + Derivative(f(x), x)*Derivative(g(x), x) - Derivative(f(x), (x, 2))

R = 2*(-Derivative(f(x), x)**2 + Derivative(f(x), x)*Derivative(g(x), x) - Derivative(f(x), (x, 

Usar las ecuaciones de lagrange no funciona en general sin meter unas dentro de otras para eliminar ciertas segundas derivadas, cambiemos a usar la fórmula de los christoffel.

In [1]:
import sympy as smp
from itertools import product

def computar(variables_str, g, g_inv):
    """
    Con la métrica como una matriz de strings y tambien la métrica inversa, calcular simbolos de christoffel, etc
    Modificado para que funcione en cualquier número de dimensiones
    """
    tau = smp.symbols("tau")

    variables = smp.symbols(variables_str, cls=smp.Function)
    variables = [v(tau) for v in variables]

    n = len(variables)
    
    for i, j in product(range(0, n), repeat=2):
        g[i][j] = smp.sympify(g[i][j])
        g_inv[i][j] = smp.sympify(g_inv[i][j])

    Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz n*n*n

    for i, j, k in product(range(0, n), repeat=3):
        for l in range(0, n):
            Gamma[i][j][k] += g_inv[i][l]*(smp.diff(g[l][k], variables[j]) + smp.diff(g[l][j], variables[k]) - smp.diff(g[j][k], variables[l]))
        Gamma[i][j][k] = smp.Rational(1, 2)*Gamma[i][j][k]
        Gamma[i][j][k] = smp.simplify(Gamma[i][j][k])
                    
    Riemman = [[[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz n*n*n*n

    for i, j, k, l in product(range(0, n), repeat=4):
        Riemman[i][j][k][l] = smp.diff(Gamma[i][l][j], variables[k]) - smp.diff(Gamma[i][k][j], variables[l])
        for alpha in range(0, n):
            Riemman[i][j][k][l] += Gamma[i][k][alpha]*Gamma[alpha][l][j] - Gamma[i][l][alpha]*Gamma[alpha][k][j]
        # https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html#trigsimp
        Riemman[i][j][k][l] = smp.simplify(Riemman[i][j][k][l])
        # Riemman[i][j][k][l] = smp.expand_trig(Riemman[i][j][k][l])
        # Riemman[i][j][k][l] = smp.trigsimp(Riemman[i][j][k][l])

    Ricci = [[0 for _ in range(n)] for _ in range(n)] # Matriz n*n

    for i, j in product(range(0, n), repeat=2):
        for alpha in range(0, n):
            Ricci[i][j] += Riemman[alpha][i][alpha][j]
        Ricci[i][j] = smp.simplify(Ricci[i][j])

    R = smp.sympify(0)

    for i, j in product(range(0, n), repeat=2):
        R += g_inv[i][j]*Ricci[i][j]
        
    R = smp.simplify(R)
    R = smp.expand_trig(R)
    R = smp.trigsimp(R)

    E = [[0 for _ in range(n)] for _ in range(n)] # Matriz n*n

    for i, j in product(range(0, n), repeat=2):
        E[i][j] = smp.simplify(Ricci[i][j] - smp.Rational(1, 2)*R*g[i][j])

    return Gamma, Riemman, Ricci, R, E

def calcular_metrica(variables_str, cambios_str, as_strings=True):
    """
    Calcula la métrica de un cambio de coordenadas aleatorio en el espacio plano. Necesita las nuevas coordenadas y las 
    viejas coordenadas en función de las nuevas. 
    Ejemplo: pasar de espacio plano con (x, y, z) a espacio plano con (r, theta, phi) y una nueva métrica se hace con:
    calcular_metrica(["r", "theta", "phi"], ["r*sin(theta)*cos(phi)", "r*sin(theta)*sin(phi)", "r*cos(theta)"])
    """
    n = len(cambios_str)
    g = [[0 for _ in range(0, n)] for _ in range(0, n)] # Matriz n*n

    cambio = [smp.sympify(v) for v in cambios_str]
    variables = smp.symbols(variables_str)

    for i, j in product(range(0, n), repeat=2):
        for v in cambio:
            # Implicitamente g_(ij) de la que venimos es la delta de kroneker
            g[i][j] += smp.diff(v, variables[i])*smp.diff(v, variables[j])
        g[i][j] = g[i][j].simplify()

    # Cambiamos las variables por variables que dependen explicitamente de tau    
    tau = smp.symbols("tau")
    variables_tau = smp.symbols(variables_str, cls=smp.Function)
    variables_tau = [v(tau) for v in variables_tau]

    for i, j in product(range(0, n), repeat=2):
        for k in range(0, n):
            g[i][j] = g[i][j].subs(variables[k], variables_tau[k])

    if as_strings:
        return [[str(g[i][j]) for j in range(0, n)] for i in range(0, n)]
    
    return g

kroneker = lambda i, j: 1 if i == j else 0

def calcular_metrica_inversa(g):
    n = len(g[0])
        
    g_symb = [[0 for _ in range(n)] for _ in range(n)] # Matriz n*n

    for i, j in product(range(0, n), repeat=2):
        g_symb[i][j] = smp.sympify(g[i][j])

    g_inversa = [[0 for _ in range(n)] for _ in range(n)] # Matriz n*n

    for i, j in product(range(0, n), repeat=2):
        g_inversa[i][j] = str((smp.Matrix(g_symb)**(-1)).row(i)[j])
    
    return g_inversa

def computar_e_imprimir(variables_str:list, g, g_inv):
    n = len(variables_str)

    Gamma, Riemman, Ricci, R, E = computar(variables_str, g, g_inv)

    v = variables_str
        
    for i, j, k in product(range(0, n), repeat=3):
        if Gamma[i][j][k] != 0:
            s = str(Gamma[i][j][k]).replace("(tau)", "")
            print(f"Gamma^{v[i]}_({v[j]} {v[k]}) = {s}")
    print()
    for i, j, k, l in product(range(0, n), repeat=4):
        if Riemman[i][j][k][l] != 0:
            s = str(Riemman[i][j][k][l]).replace("(tau)", "")
            print(f"Riemman^{v[i]}_({v[j]} {v[k]} {v[l]}) = {s}")
    print()
    for i, j in product(range(0, n), repeat=2):
        if Ricci[i][j] != 0:
            s = str(Ricci[i][j]).replace("(tau)", "")
            print(f"Ricci_({v[i]} {v[j]}) = {s}")
    print()
    print("R =", str(R).replace("(tau)", ""))
    print()
    for i, j in product(range(0, n), repeat=2):
        if E[i][j] != 0:
            s = str(E[i][j]).replace("(tau)", "")
            print(f"E_({v[i]} {v[j]}) = {s}")

In [2]:
# comprobación ejercicio 9
variables = ["u", "v", "w", "z"]

g = [
    ["0", "1", "0", "0"],
    ["1", "0", "0", "0"],
    ["0", "0", "f(v(tau))", "1"],
    ["0", "0", "1", "0"]
]

g_inversa = [
    ["0", "1", "0", "0"],
    ["1", "0", "0", "0"],
    ["0", "0", "0", "1"],
    ["0", "0", "1", "-f(v(tau))"]
]

computar_e_imprimir(variables, g, g_inversa)

Gamma^u_(w w) = -Derivative(f(v), v)/2
Gamma^z_(v w) = Derivative(f(v), v)/2
Gamma^z_(w v) = Derivative(f(v), v)/2

Riemman^u_(w v w) = -Derivative(f(v), (v, 2))/2
Riemman^u_(w w v) = Derivative(f(v), (v, 2))/2
Riemman^z_(v v w) = Derivative(f(v), (v, 2))/2
Riemman^z_(v w v) = -Derivative(f(v), (v, 2))/2


R = 0



In [3]:
# Comprobación ejercicio 2
variables = ["t", "r", "phi", "z"]

g = [
    ["-(1-w^2*r(tau)^2)", "0", "-w*r(tau)^2", "0"],
    ["0", "1", "0", "0"],
    ["-w*r(tau)^2", "0", "r(tau)^2", "0"],
    ["0", "0", "0", "1"]
]

g_inversa = calcular_metrica_inversa(g)

computar_e_imprimir(variables, g, g_inversa)

Gamma^r_(t t) = -w**2*r
Gamma^r_(t phi) = w*r
Gamma^r_(phi t) = w*r
Gamma^r_(phi phi) = -r
Gamma^phi_(t r) = -w/r
Gamma^phi_(r t) = -w/r
Gamma^phi_(r phi) = 1/r
Gamma^phi_(phi r) = 1/r



R = 0



In [4]:
# Comprobación ejercicio 8
variables = ["u", "r", "x", "y"]

g = [
    ["H(r(tau))", "-1", "0", "0"],
    ["-1", "0", "0", "0"],
    ["0", "0", "r(tau)^2", "0"],
    ["0", "0", "0", "r(tau)^2"]
]

g_inversa = calcular_metrica_inversa(g)

computar_e_imprimir(variables, g, g_inversa)

Gamma^u_(u u) = Derivative(H(r), r)/2
Gamma^u_(x x) = r
Gamma^u_(y y) = r
Gamma^r_(u u) = H(r)*Derivative(H(r), r)/2
Gamma^r_(u r) = -Derivative(H(r), r)/2
Gamma^r_(r u) = -Derivative(H(r), r)/2
Gamma^r_(x x) = H(r)*r
Gamma^r_(y y) = H(r)*r
Gamma^x_(r x) = 1/r
Gamma^x_(x r) = 1/r
Gamma^y_(r y) = 1/r
Gamma^y_(y r) = 1/r

Riemman^u_(u u r) = -Derivative(H(r), (r, 2))/2
Riemman^u_(u r u) = Derivative(H(r), (r, 2))/2
Riemman^u_(x u x) = r*Derivative(H(r), r)/2
Riemman^u_(x x u) = -r*Derivative(H(r), r)/2
Riemman^u_(y u y) = r*Derivative(H(r), r)/2
Riemman^u_(y y u) = -r*Derivative(H(r), r)/2
Riemman^r_(u u r) = -H(r)*Derivative(H(r), (r, 2))/2
Riemman^r_(u r u) = H(r)*Derivative(H(r), (r, 2))/2
Riemman^r_(r u r) = Derivative(H(r), (r, 2))/2
Riemman^r_(r r u) = -Derivative(H(r), (r, 2))/2
Riemman^r_(x r x) = r*Derivative(H(r), r)/2
Riemman^r_(x x r) = -r*Derivative(H(r), r)/2
Riemman^r_(y r y) = r*Derivative(H(r), r)/2
Riemman^r_(y y r) = -r*Derivative(H(r), r)/2
Riemman^x_(u u x) = -H(r)*D